# **Walmart Recruiting - Store Sales Forecasting**

We are provided historical sales data for 45 Walmart stores located in different regions. Each store contains a number of departments, and we are tasked with predicting the department-wide sales for each store.

In addition, Walmart runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks. Part of the challenge presented by this competition is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data.

# Overview
The dataset essentially contains five scv files which are descibed as follows.

**stores.csv:**

  This file contains anonymized information about the 45 stores, indicating      the type and size of store.

**train.csv:**

  This is the historical training data, which covers to 2010-02-05 to 2012-11-01. Within this file we will find the following fields:

  - Store - the store number
  - Dept - the department number
  - Date - the week
  - Weekly_Sales -  sales for the given department in the given store
  - IsHoliday - whether the week is a special holiday week

**test.csv:**

 This file is identical to train.csv, except we have withheld the weekly sales. We must predict the sales for each triplet of store, department, and date in this file.

**features.csv:**

 This file contains additional data related to the store, department, and regional activity for the given dates. It contains the following fields:

  - Store - the store number
  - Date - the week
  - Temperature - average temperature in the region
  - Fuel_Price - cost of fuel in the region
  - MarkDown1 to MarkDown5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
  - CPI - the consumer price index
  - Unemployment - the unemployment rate
  - IsHoliday - whether the week is a special holiday week

**sampleSubmission.csv:**

We have been provided with dates where we can predict the salesfor each date mentioned in this csv file.

For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

  - Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
  -Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
  - Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
  - Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

**Evaluation:**

 This competition is evaluated on the weighted mean absolute error (WMAE):
 >$WMAE = [1/(sum_{i=1}^n w_i]) \sum_{i=1}^n \ w_i|y_i - y_i^{hat}|$

 where
 * n is the number of rows
 * $y_i^{hat}$  is the predicted sales
 * $y_i$  is the actual sales
 * $w_i$  are weights, w = 5 if the week is a holiday week, 1 otherwise


## Import libraries

In [87]:
# importing all the libraries needed 
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
sns.set_style("whitegrid")
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, mean_absolute_error

#For date time functions
from datetime import datetime
from datetime import timedelta
import math

# Importing the most popular regression libraries.
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, ridge_regression, Lasso, SGDRegressor, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [88]:
# auto-arima library comes under pyramid utility function, hence installing pyramid library.
%pip install pmdarima
import pmdarima

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# !apt install unzip
# !unzip  walmart-recruiting-store-sales-forecasting.zip

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Load Dataset

In [ ]:
data_dir = 'E:/Work_DatPT/Study/business-intelligence-analytics/data/'

In [ ]:
#Loading the data from csv files.
train = pd.read_csv(data_dir + 'train.csv')
features = pd.read_csv(data_dir + 'features.csv')
stores = pd.read_csv(data_dir + 'stores.csv')

In [ ]:
train

In [ ]:
features

In [ ]:
stores.head()

# Data Cleanning

## Check data

In [ ]:
#Check for null values in train
train.isnull().any()

In [ ]:
#Check for null values in features.
features.isnull().any()

In [ ]:
#Check for null values in stores.
stores.isnull().any()

There are null values present in below columns.
- MarkDown1, MarkDown2, MarkDown3, MarkDown4, MarkDown5, CPI and Unemployment


In [ ]:
#Count of null values present in respective columns of features.
features.isnull().sum()

In [ ]:
#Original shape of train, features and stores data.
print('train: ', train.shape)
print('features: ', features.shape)
print('stores ', stores.shape)

## Merge data

In [ ]:
#Merging the three csv files using inner join.
data = train.merge(features, on=['Store', 'Date'], how='inner').merge(stores, on=['Store'], how='inner')
print(data.shape)

In [ ]:
data.head()

In [ ]:
# Removing additional IsHoliday column (IsHoliday_y) and renaming original IsHoliday_x column to IsHoliday.
data = data.drop(['IsHoliday_y'], axis=1)
data = data.rename(columns={'IsHoliday_x':'IsHoliday'})
data.columns

In [ ]:
data.head()

In [ ]:
data.shape

## Dense

In [ ]:
import missingno as msno
msno.matrix(data)

## Handle missing values

NaN for markdown means that there was no markdown event for that date. So we can replace that with 0 indicating no mark down

In [ ]:
# First we check what happens when we replace NaN's with 0.
data.fillna(0).head()

There are null values present in the dataset. Let's remove those.

In [ ]:
data.isnull().head()

In [ ]:
data.isna().sum()

In [ ]:
# Removing rows with null values in all columns
data.dropna(axis=0, how="all", inplace=True)
# Removing all rows with null values in all rows
data.dropna(axis=1, how="all", inplace=True)

In [ ]:
data.head()

In [ ]:
# Fill missing values with 0
data = data.fillna(0)

In [ ]:
data.isna().sum()

In [ ]:
data.describe()

In [ ]:
print('Final Data Shape: ', data.shape)

In [ ]:
# # Taking Mean of Temparature, Weekly Sales and Unemployment columns and forming additional columns.
# temp_mean = data.Temperature.fillna(data['Temperature'].mean())
# unemployment_mean = data.Unemployment.fillna(data['Unemployment'].mean())
# weekly_sales_mean = data.Weekly_Sales.fillna(data['Weekly_Sales'].mean())

# data['Weekly Sales Mean'] = weekly_sales_mean
# data['Unemployment Mean'] = unemployment_mean
# data['Temperature Mean']  = temp_mean

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
# Remove negative values as sales cannot be negative values.
data = data[data['Weekly_Sales'] >= 0]
data.shape

# Exploratory Data Analysis



## 1.Weekly Sales by Department

In [ ]:
#Plot of Weekly Sales and Department.
x = data['Dept']
y = data['Weekly_Sales']
plt.figure(figsize=(15,5))
plt.title('Weekly Sales by Department')
plt.xlabel('Departments')
plt.ylabel('Weekly Sales')
plt.plot(x,y)
plt.show()

#Plot of Mean Weekly Sales and Department.
x = data['Dept']
y = data['Weekly Sales Mean']
plt.figure(figsize=(15,5))
plt.title('Imputed Weekly Sales by Department')
plt.xlabel('Departments')
plt.ylabel('Imputed Weekly Sales')
plt.scatter(x,y)
plt.show()

 - Observation: Most of the Departments have weekly sales below 200000.

## 2.Weekly Sales by Stores

In [ ]:
#Plot of Weekly Sales and Stores.
x = data['Store']
y = data['Weekly_Sales']
plt.figure(figsize=(15,5))
plt.title('Weekly Sales by Stores')
plt.xlabel('Store Numbers')
plt.ylabel('Weekly Sales')
plt.plot(x,y)
plt.show()

#Plot of Mean Weekly Sales and Stores.
x = data['Store']
y = data['Weekly Sales Mean']
plt.figure(figsize=(15,5))
plt.title('Imputed Weekly Sales by Stores')
plt.xlabel('Store Numbers')
plt.ylabel('Imputed Weekly Sales')
plt.scatter(x,y)
plt.show()

 - Observation: Weekly sales is highest for Store number 10 and lowest for Store number 33.

## 3.Weekly Sales by Year, Month and Week

In [ ]:
# First we need to get Year, Month and Week columns from Date column.
data.Date = pd.to_datetime(data.Date)
data['Year']  = data.Date.dt.year
data['Month'] = data.Date.dt.month
data['Week']  = data.Date.dt.week

data.head(2)

In [ ]:
# Plot of Weekly Sales and Year.
x = data['Year']
y = data['Weekly_Sales']
plt.figure(figsize=(15,5))
plt.title('Weekly Sales by Year')
plt.xlabel('Year')
plt.ylabel('Weekly Sales')
plt.plot(x,y)
plt.show()

In [ ]:
data['Year'][100000:110000]

 - Observation: Year 2011 had the highest weekly sales.

In [ ]:
# Plot of Weekly Sales and Month.
x = data['Month']
y = data['Weekly_Sales']
plt.figure(figsize=(15,5))
plt.title('Weekly Sales by Month')
plt.xlabel('Month')
plt.ylabel('Weekly Sales')
plt.plot(x,y)
plt.show()

 - Observation: November month witnessed the maximum weekly sales.

In [ ]:
# Plot of Weekly Sales and Week.
x = data['Week']
y = data['Weekly_Sales']
plt.figure(figsize=(15,5))
plt.title('Weekly Sales by Week')
plt.xlabel('Week Number')
plt.ylabel('Weekly Sales')
plt.plot(x,y)
plt.show()

 - Observation: Week 47 of November month had the highest weekly sales.

## 4.Store Size by Store Type

- Let's make a pie chart to show the ratio of A, B, and C types of total 45 Walmart stores.
- First, let's group data by type of stores and see the descriptive figures.
- Later we will plot box plot for Store Type and Store Size.

In [ ]:
print("the shape of stores data set is", stores.shape)
print('='*50)
print("the unique value of store is", stores['Store'].unique())
print('='*110)
print("the unique value of Type is", stores['Type'].unique())

In [ ]:
sorted_type = stores.groupby('Type')
print(sorted_type.describe()['Size'].round(2))

In [ ]:
# Make Pie chart for Stores including Weekly Sales.
plt.style.use('ggplot')
labels=['A store','B store','C store']
sizes=sorted_type.describe()['Size'].round(1)
sizes=[(22/(17+6+22))*100,(17/(17+6+22))*100,(6/(17+6+22))*100] # convert to the proportion

fig, axes = plt.subplots(1,1, figsize=(10,10))

wprops={'edgecolor':'black',
      'linewidth':2}

tprops = {'fontsize':30}

axes.pie(sizes,
        labels=labels,
        explode=(0.0,0,0),
        autopct='%1.1f%%',
        pctdistance=0.6,
        labeldistance=1.2,
        wedgeprops=wprops,
        textprops=tprops,
        radius=0.8,
        center=(0.5,0.5))
plt.show()

In [ ]:
# Box Plot of Store Type and Store Size.
type_size = pd.concat([stores['Type'], stores['Size']], axis=1)
plt.figure(figsize=(8,6))
fig = sns.boxplot(x='Type', y='Size', data=type_size, showfliers=False)

Observations:
- Type A store is the largest store and C being the smallest.
- There is considerable separation among the store types, hence store type is best predictor for store size.


## 5.Train-Stores table analysis

In [ ]:
train_stores = train.merge(stores, on='Store', how='inner')
train_stores.head()

In [ ]:
# Form Date, Year, Month, Week, Day and No. of days.
train_stores['Date'] = pd.to_datetime(train_stores['Date'])
train_stores['Year'] = train_stores['Date'].dt.year
train_stores['Month']= train_stores['Date'].dt.month
train_stores['Week'] = train_stores['Date'].dt.week
train_stores['Day']  = train_stores['Date'].dt.day
train_stores['No. of days'] = (train_stores['Date'].dt.date - train_stores['Date'].dt.date.min()).apply(lambda x:x.days)

In [ ]:
train_stores.head()

In [ ]:
Year        = pd.Series(train_stores['Year'].unique())
Week        = pd.Series(train_stores['Week'].unique())
Month       = pd.Series(train_stores['Month'].unique())
Day         = pd.Series(train_stores['Day'].unique())
No_of_days  = pd.Series(train_stores['No. of days'].unique())

## 6.Weekly Sales for Store Type

In [ ]:
# There are negative values present in Weekly sales which are absurd because sales cannot be negative.
train_stores= train_stores[train_stores['Weekly_Sales'] > 0]

In [ ]:
# Plot of Store Type and Weekly Sales
type_sales = pd.concat([train_stores['Type'], train_stores['Weekly_Sales']], axis=1)
plt.figure(figsize=(8,6))
plt.title('Box Plot of Store Type and Weekly Sales')
fig = sns.boxplot(x='Type', y='Weekly_Sales', data=type_sales, showfliers=False)

 - Observation: Type A stores have their medians higher than any other medians in other store types, so the weekly sales for store type A is more than other store types.

## 7.Weekly Sales for Store Size

In [ ]:
# Plot of Store Size and Weekly Sales.
size_sales = pd.concat([train_stores['Size'], train_stores['Weekly_Sales']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Store Size and Weekly Sales')
fig = sns.boxplot(x='Size', y='Weekly_Sales', data=size_sales, showfliers=False)

 - Observation: There is no clear distinction that can be drawn from this plot.

## 8.Weekly Sales by Store Number and Store Type

In [ ]:
# Plot of Weekly Sales by Store Number and Store Type.
data_8 = pd.concat([train_stores['Store'], train_stores['Weekly_Sales'], train_stores['Type']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Weekly Sales by Store Number and Store Type')
fig = sns.boxplot(x='Store', y='Weekly_Sales', data=data_8, showfliers=False, hue='Type')

 - Observation: We can observe that store number 4 followed by 20 have the highest sales, so are their median sales values.

## 9.Weekly Sales by Store Number and Holiday

In [ ]:
# Plot of Weekly Sales by Store Number and Holiday.
data_9 = pd.concat([train_stores['Store'], train_stores['Weekly_Sales'], train_stores['IsHoliday']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Weekly Sales by Store Number and Holiday')
fig = sns.boxplot(x='Store', y='Weekly_Sales', data=data_9, showfliers=False, hue='IsHoliday')

 - Observation: We can't interpret the reation here, but we can observe that sales are more on holidays.

## 10.Weekly Sales by Department and Store Type

In [ ]:
data_10 = pd.concat([train_stores['Dept'], train_stores['Weekly_Sales'], train_stores['Type']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Weekly Sales by Department and Store Type')
fig = sns.boxplot(x='Dept', y='Weekly_Sales', data=data_10, showfliers=False, hue='Type')

 - Observation: Here we can infer that weekly sales for store type A is more than any other store types.

## 11.Weekly Sales by Department and Holiday

In [ ]:
data_11= pd.concat([train_stores['Dept'], train_stores['Weekly_Sales'], train_stores['IsHoliday']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Weekly Sales by Department and Holiday')
fig = sns.boxplot(x='Dept', y='Weekly_Sales', data=data_11, showfliers=False, hue="IsHoliday")

Observations:
- Department number 92 has highest sales that too happened on holiday.
- There is no explicit relation between Department and Weekly Sales.

## 12.Weekly Sales on Holidays and Non-Holidays

In [ ]:
holiday = train_stores['Weekly_Sales'].loc[train_stores['IsHoliday']== True] # Weekly Sales in Holidays
non_holiday = train_stores['Weekly_Sales'].loc[train_stores['IsHoliday']== False] #Weekly Sales in Non-holidays.

sns.barplot(x='IsHoliday', y='Weekly_Sales', data=train_stores)

 - Observation: We see the sales are higher on holidays than on non holidays.

## 13.Monthwise Weekly Sales

In [ ]:
data_13 = pd.concat([train_stores['Month'], train_stores['Weekly_Sales']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Monthwise Weekly Sales')
fig = sns.boxplot(x='Month', y='Weekly_Sales', data=data_13, showfliers=False)

 - Observation: December month had maximum sales.

## 14.Weekly Sales by Month and Holiday

In [ ]:
data_14 = pd.concat([train_stores['Month'], train_stores['Weekly_Sales'], train_stores['IsHoliday']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Weekly Sales by Month and Holiday')
fig = sns.boxplot(x='Month', y='Weekly_Sales', data=data_14, showfliers=False, hue='IsHoliday')

 - Observation: Non holidays have sales in each of the monhs and in December the sales, which is the highest among all sales, is even more than sales in holidays.

## 15.Weekly Sales by Month and Store Type

In [ ]:
data_15 = pd.concat([train_stores['Month'], train_stores['Weekly_Sales'], train_stores['Type']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Weekly Sales by Month and Store Type')
fig = sns.boxplot(x='Month', y='Weekly_Sales', data=data_15, showfliers=False, hue='Type')

 - Observation: In every month Store Type A has the maximum sales.

## 16.Weekly Sales by Year

In [ ]:
data_16 = pd.concat([train_stores['Year'], train_stores['Weekly_Sales']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Weekly Sales by Year')
fig = sns.boxplot(x='Year', y='Weekly_Sales', data=data_16, showfliers=False)

 - Observation: There seems no clear distinction from this plot.

## 17.Weekly Sales by Week

In [ ]:
data_17 = pd.concat([train_stores['Week'], train_stores['Weekly_Sales']], axis=1)
plt.figure(figsize=(20,6))
plt.title('Box Plot of Weekly Sales by Week')
fig = sns.boxplot(x='Week', y='Weekly_Sales', data=data_17, showfliers=False)

 - Observation: The 51st week i.e. third week of December has the highest sales.

## 18.Histogram of Weekly Sales

In [ ]:
plt.figure(figsize=(20,6))
plt.title('Histogram of Weekly Sales')
fig = sns.distplot(train_stores['Weekly_Sales'].dropna()) #Taking only valid weekly sales values.
print('Minimum sales:', train_stores['Weekly_Sales'].min())

 - Observation: About all sales have happened below 100000. Maximum sales are done at sales value of 1000.

## 19.Probability Plot of Weekly Sales

In [ ]:
plt.figure(figsize=(10,6))
fig = stats.probplot(train_stores['Weekly_Sales'], plot=plt)

## 20.Distributions of 5 MarkDowns

In [ ]:
# Histograms of MarkDowns.
def markdowns(data, column):
  plt.figure(figsize=(20,6))
  sns.distplot(data[column], kde=True)
  plt.title(str(column)+' distribution')
  plt.xlabel(column)
  
markdowns(data, 'MarkDown1')
markdowns(data, 'MarkDown2')
markdowns(data, 'MarkDown3')
markdowns(data, 'MarkDown4')
markdowns(data, 'MarkDown5')

In [ ]:
# Scatter plots of MarkDowns.
def scatter(data, column):
    plt.figure(figsize=(20,6))
    plt.scatter(data[column] , data['Weekly_Sales'])
    plt.title('Weekly sales by '+str(column))
    plt.ylabel('Weekly_Sales')
    plt.xlabel(column)
    

In [ ]:
scatter(data, 'MarkDown1')
scatter(data, 'MarkDown2')
scatter(data, 'MarkDown3')
scatter(data, 'MarkDown4')
scatter(data, 'MarkDown5')

  - Observation: Most of the Markdowns have been given for Weekly sales of less than 200000

## 21.Fuel, Temparature and CPI effects

In [ ]:
scatter(data, 'Fuel_Price') # Fuel
scatter(data, 'Temperature') #Temparature
scatter(data, 'CPI') #CPI

- Observations:
  1. Most of the Fuel price value lies in range of 2.5 to 4 for weekly sales of less than 200000.
  2. Sales of less than 200000 happened for average temparature range of 30 to 80.
  3. No weekly sales for CPI range of 145 to 180. CPI range (200 - 230) is more when the weekly sales is less than 200000.
  

## Compute Correlation Matrix

In [ ]:
sns.set_style('dark')
corr = data.corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)]=True

f,ax = plt.subplots(figsize=(20,15))
cmap = sns.diverging_palette(220, 10, as_cmap=True)

fig = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=True,
            linewidths=.2, cbar_kws={"shrink": .5})

## Logical deductions
 - If the Fuel Price in the area is less then more customers would travel to the store for shopping purpose, however that has a very less correlation.
 - If there is an inflation, Cost of goods will rise which causes CPI to increase. This will cause a reduction in sales which is obvious. But again, CPI doesn't have much correlation.
 - For a lesser store numbers, the weekly sales are higher. Unemployment, Temperature is less, Sales are higher. (Not much correlation)

# Time Series Models

## 1.ARIMA

ARIMA stands for Autoregressive Integrated Moving Average model. Univariate (single vector) ARIMA is a forecasting technique that projects the future values of a series based entirely on its own inertia. Below steps  will compute forecasting values of the input univariate data using ARIMA.
   - Load the data.
   - Visualize the available univariate data in timely fashion.
   - Look at the seasonal decomposition parameter to check for seasonality, trend etc. available or not.
   - Perform a test of stationarity using one of the time-series stationarity methods.
    - If the time series is non-stationary, make it stationary. 
    - If the time series is staionary then no need to make it stationary.
   - Split the univariate data in train and test.
   - The most important step is to find the appropriate values of triplet p,d,q, where p is number of Auto Regressors, d is Integration or difference and q is number of Moving Average by plotting ACF and PACF graphs.
   - Once got the values of p,d,q, fit ARIMA model on train data.
   - Forecast the test data values and plot the visualization of train, test and predicted test data values on single graph.
   - Calculate Root mean squared error (RMSE) on predicted values.

In [ ]:
# It is necessary to have the date columns present in the dataset to be available in the datetime format as in ARIMA model it is required to see the sales values in date-wise fashion.
# Hence converting the string formatted Date into datetime format.
data.Date = pd.to_datetime(data.Date,format='%Y-%m-%d')
data.index = data.Date
data = data.drop('Date', axis=1)

In [ ]:
data = data.resample('MS').mean() # Resampling the time series data with month starting first.

In [ ]:
# Train-Test splitting of time series data
train_data = data[:int(0.7*(len(data)))]
test_data = data[int(0.7*(len(data))):]

print('Train:', train_data.shape)
print('Test:', test_data.shape)

In [ ]:
print('Train data:\n')
print(train_data.tail())
print('='*50,'\n')
print('Test data:\n')
print(test_data.head())

In [ ]:
# ARIMA takes univariate data.
train_data = train_data['Weekly_Sales']
test_data = test_data['Weekly_Sales']

In [ ]:
# Plot of Weekly_Sales with respect to years in train and test.
train_data.plot(figsize=(20,8), title= 'Weekly_Sales', fontsize=14)
test_data.plot(figsize=(20,8), title= 'Weekly_Sales', fontsize=14)
plt.show()

In [ ]:
# Decomposition of time series data. It is necessary to see whether the trend, seasonality and residual are present in data or not.
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data['Weekly_Sales'], model='additive')
result.plot()
plt.show()

- Observation:
  - The trend of weekly sales is such that it first decreases then elevates.
  - Given walmart dataset is seasonal dataset as it is repeating the same pattern in the month of Nov-Dec.

**Dickey-Fuller Test for Stationarity**

  - Officially, this is called the ‘augmented Dickey-Fuller test’, but most folks just say ‘Dickey-Fuller’ when talking about it.  This is a test that tests the null hypothesis that a unit root is present in time series data.    To make things a bit more clear, this test is checking for stationarity or non-stationary data.  The test is trying to reject the null hypothesis that a unit root exists and the data is non-stationary. If the null hypothesis is rejected, then the alternate can be considered valid (e.g., the data is stationary).  
  - When you run the test, you’ll get an ADF value and a p-value. The ADF number should be a negative number and the p-value should be beneath a certain threshold value (e.g., 1% or 5%, etc) for a confidence level. Here, we'll use 5% (or 95% confidence level), so if the p-value is greater than 0.05 then we say we fail to reject the null hypothesis because the data has a unit root and is non-stationary.  If the p-value is less than or equal to 0.05, we can say we reject the null hypothesis because the data does not have a unit root and is stationary.

In [ ]:
# A check of sationarity of data using Dicky-Fuller test.
from statsmodels.tsa.stattools import adfuller
result = adfuller(data['Weekly_Sales'])
print('ADF Statistic: {}'.format(result[0]))
print('p-value: {}'.format(result[1]))
print('Critical Values:')
for key, value in result[4].items():
    print('\t{}: {}'.format(key, value))

 - As we can see that our p-value is definitely less than 0.5 and is even less than 0.01 so we can say with pretty good confidence that we can reject the null (unit root, non-stationary data) and can assume our data is stationary. Additionally, our ADF is much less than our 1% confidence value of -3.43, so we have another confirmation that we can reject the null.

**Perform Auto-ARIMA:**
  - We used Auto-ARIMA over ARIMA because in ARIMA model we are required to find appropriate values of number of Auto Regressor (p), number of Moving Average (q), and Integrated (difference, d), which is tedious and time consuming. In Auti-ARIMA we get to see the triplet (p,q,d) values automatically which more importantly best fits the model. Hence we used Auto-ARIMA over ARIMA model.

In [ ]:
# Applying auto_arima model on train data.
#model_auto_arima = auto_arima(train_data, trace=True, error_action='ignore', suppress_warnings=True)
model_auto_arima = pmdarima.auto_arima(train_data, trace=True,start_p=0, start_q=0, start_P=0, start_Q=0,
                  max_p=10, max_q=10, max_P=10, max_Q=10, seasonal=True,
                  stepwise=False, suppress_warnings=True, D=1, max_D=10,
                  error_action='ignore',approximation = False)
model_auto_arima.fit(train_data)

In [ ]:
# Predicting the test values using predict function.
forecast = model_auto_arima.predict(n_periods=len(test_data))
forecast = pd.DataFrame(forecast,index = test_data.index,columns=['Prediction'])
plt.figure(figsize=(20,6))
plt.title('Prediction of Weekly Sales using Auto ARIMA model', fontsize=20)
plt.plot(train_data, label='Train')
plt.plot(test_data, label='Test')
plt.plot(forecast, label='Prediction using ARIMA Model')
plt.legend(loc='best')
plt.xlabel('Date', fontsize=14)
plt.ylabel('Weekly Sales', fontsize=14)
plt.show()

In [ ]:
# Performance metric for ARIMA model -MSE/RMSE
print('Mean Squared Error (MSE) of ARIMA: ', mean_squared_error(test_data, forecast))
print('Root Mean Squared Error (RMSE) of ARIMA: ', math.sqrt(mean_squared_error(test_data, forecast)))
print('Mean Absolute Deviation (MAD) of ARIMA: ', mean_absolute_error(test_data, forecast))

In [ ]:
# arima_train = train.sort_values(by='Date', ascending=True) # Sorting the data in increasing order of Date and then splitting.
# y = arima_train['Weekly_Sales']
# X = arima_train.drop(['Weekly_Sales'], axis=1)

# def wmae_arima(test, pred): # WMAE for train
#   weights = X['IsHoliday'].apply(lambda is_holiday:5 if is_holiday else 1)
#   error = np.sum(weights * np.abs(test - pred), axis=0) / np.sum(weights)
#   return error

# print('WMAE of ARIMA: ', wmae_arima(X, y))

## 2.Holt Winters Model

Holt-Winters is a time-series model and is a way to model three aspects of the time series: a typical value (average), a slope (trend) over time, and a cyclical repeating pattern (seasonality). Holt-Winters uses exponential smoothing to encode lots of values from the past and use them to predict “typical” values for the present and future. Below steps are required to use Holt-Winters model.
 - Load the data.
 - Visualize the available univariate data in timely fashion.
 - Fit the model using Holt-Winters method on train data.
 - Predict the values using of test data. 
 - Visualize the training data, test data and predicted data on single graph.
 - Calculate Root Mean Squared Error (RMSE) on predicted data..


In [ ]:
# Fitting the Holt-Winters method for Weekly Sales.
from statsmodels.tsa.api import ExponentialSmoothing
model_holt_winters = ExponentialSmoothing(train_data, seasonal_periods=7, trend='additive', seasonal='additive').fit() #Taking additive trend and seasonality.
pred = model_holt_winters.forecast(len(test_data))# Predict the test data

#Visualize train, test and predicted data.
plt.figure(figsize=(20,6))
plt.title('Prediction of Weekly Sales using Holt-Winters model', fontsize=20)
plt.plot(train_data, label='Train')
plt.plot(test_data, label='Test')
plt.plot(pred, label='Prediction using Holt Winters Methods')
plt.legend(loc='best')
plt.xlabel('Date', fontsize=14)
plt.ylabel('Weekly Sales', fontsize=14)
plt.show()

In [ ]:
# Performance metric for Holt-Winters model -MSE/RMSE
print('Mean Squared Error (MSE) of Holt-Winters: ', mean_squared_error(test_data, pred))
print('Root Mean Squared Error (RMSE) of Holt-Winters: ', math.sqrt(mean_squared_error(test_data, pred)))
print('Mean Absolute Deviation (MAD) of Holt-Winters: ', mean_absolute_error(test_data, pred))

# Data Pre-processing

## Standard 

In [ ]:
train_ = train

In [ ]:
#Joining the train data with store and features data using inner join.
train = train.merge(features, on=['Store', 'Date'], how='inner').merge(stores, on=['Store'], how='inner')
print(train.shape)

In [ ]:

# Make one IsHoliday column instead of two.
train = train.drop(['IsHoliday_y'], axis=1)
train = train.rename(columns={'IsHoliday_x':'IsHoliday'})
print('Train columns:\n', train.columns)

### Extract Date

In [ ]:
# Converting Date to datetime
train['Date'] = pd.to_datetime(train['Date'])

# Extract date features
train['Date_dayofweek'] = train['Date'].dt.dayofweek
train['Date_month'] = train['Date'].dt.month 
train['Date_year'] = train['Date'].dt.year
train['Date_day'] = train['Date'].dt.day 

### Numeric Encoding

In [ ]:
train_data = [train]

# Converting Categorical Variable 'Type' into Numerical Variables.
type_mapping = {"A": 1, "B": 2, "C": 3}
for dataset in train_data:
    dataset['Type'] = dataset['Type'].map(type_mapping)

# Converting Categorical Variable 'IsHoliday' into Numerical Variables.
type_mapping = {False: 0, True: 1}
for dataset in train_data:
    dataset['IsHoliday'] = dataset['IsHoliday'].map(type_mapping)

### New holidays -  Super Bowl, Labor Day, Thanksgiving and Christmas

In [ ]:
# Kaggle has provided some dates to be allocated to special holidays. We have taken the special holidays into account and marked them as holidays.
train['Super_Bowl'] = np.where((train['Date'] == datetime(2010,2,10)) | (train['Date'] == datetime(2011,2,11)) | 
                               (train['Date'] == datetime(2012,2,10)) | (train['Date'] == datetime(2013,2,8)), 1, 0)
train['Labor_day'] = np.where((train['Date'] == datetime(2010,9,10)) | (train['Date'] == datetime(2011,9,9)) | 
                              (train['Date'] == datetime(2012,9,7)) | (train['Date'] == datetime(2013,9,6)), 1, 0)
train['Thanksgiving'] = np.where((train['Date']==datetime(2010, 11, 26)) | (train['Date']==datetime(2011, 11, 25)) | 
                                 (train['Date']==datetime(2012, 11, 23)) | (train['Date']==datetime(2013, 11, 29)),1,0)
train['Christmas'] = np.where((train['Date']==datetime(2010, 12, 31)) | (train['Date']==datetime(2011, 12, 30)) | 
                              (train['Date']==datetime(2012, 12, 28)) | (train['Date']==datetime(2013, 12, 27)),1,0)

In [ ]:
print('Train holidays:\n')
print ('Christmas:\n', train.Christmas.value_counts(),'\n')
print ('Super Bowl:\n', train.Super_Bowl.value_counts(),'\n')
print ('Thanksgiving:\n', train.Thanksgiving.value_counts(),'\n')
print ('Labor Day:\n', train.Labor_day.value_counts(),'\n')

In [ ]:
# Since we have Imputed IsHoliday according to Extra holidays..These extra holiday variable has redundant.
# Droping the Extra holiday variables because its redundant.
dp = ['Super_Bowl','Labor_day','Thanksgiving','Christmas']
train.drop(dp, axis=1, inplace=True)

In [ ]:
train = train.fillna(0)
#Remove negative values as sales cannot be negative.
train = train[train['Weekly_Sales'] >= 0]
train.shape

In [ ]:
train.info()

## Remove unnecessary columns

In [ ]:
# Not so important features.
# features_drop=['Unemployment','CPI','MarkDown5']
features_drop = ['Date_year','CPI','MarkDown1']
train_imp = train.drop(features_drop, axis=1)

print('Final train shape:', train_imp.shape)
train_imp.head(2)

## Train-Test splitting

In [ ]:
train_imp = train_imp.sort_values(by='Date', ascending=True) # Sorting the data in increasing order of Date and then splitting.
y = train_imp['Weekly_Sales']
X = train_imp.drop(['Weekly_Sales'], axis=1) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # Train:Test = 70:30 splitting.
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.3) #Train:CV = 70:30 splitting.

# Remove Date column as it does not allow the models to fit on the data. 
X_train = X_train.drop(['Date'], axis=1)
X_cv = X_cv.drop(['Date'], axis=1)
X_test = X_test.drop(['Date'], axis=1)

# Final shapes.
print('Train:', X_train.shape, y_train.shape)
print('CV:', X_cv.shape, y_cv.shape)
print('Test', X_test.shape, y_test.shape)

In [ ]:
"""Define Performance metric - Weighted Mean Absolute Error (WMAE)"""

def wmae_train(test, pred): # WMAE for train
  weights = X_train['IsHoliday'].apply(lambda is_holiday:5 if is_holiday else 1)
  error = np.sum(weights * np.abs(test - pred), axis=0) / np.sum(weights)
  return error  

def wmae_cv(test, pred): # WMAE for CV
  weights = X_cv['IsHoliday'].apply(lambda is_holiday:5 if is_holiday else 1)
  error = np.sum(weights * np.abs(test - pred), axis=0) / np.sum(weights)
  return error

def wmae_test(test, pred): # WMAE for test
  weights = X_test['IsHoliday'].apply(lambda is_holiday:5 if is_holiday else 1)
  error = np.sum(weights * np.abs(test - pred), axis=0) / np.sum(weights)
  return error

# Apply Regreesion Models

Here we will use below regression models for Weekly Sales Prediction.
  - Linear Reagression
  - K Nearest Neighbors Regression
  - Ridge Regression
  - Lasso Regression
  - Decision Tree Regression
  - Random Forest Regression
  - ExtraTrees Regression
  - XGBoost Regression

In every model below steps will be performed.
  - Define the parameters that each library takes.
  - Fit the model on training data.
  - Hyperparameter-tune the parameters using GridSearch/RandomSearch technique.
  - Retrain the model on tuned paramteres.
  - Predict the test values.
  - Get the accuracy using score function.


## 1.Linear Regression

In [ ]:
# # Define list of empty train error and cv error.
# error_cv_lr = []
# error_train_lr = []
# fit_intercept = [True,False]
# normalize = [True,False]
# lr_hyperparams = []

# """Calculating train and CV errors for Fit Intercept and Normalize parameters."""

# for i in fit_intercept:
#     for j in normalize:
#         lr = LinearRegression(fit_intercept=i, normalize=j) # Apply Linear Regression.
#         lr.fit(X_train, y_train) # Fit the model.
#         y_pred_cv_lr = lr.predict(X_cv) # Predict CV data.
#         y_pred_train_lr = lr.predict(X_train) # Predict Train data.
#         error_cv_lr.append(wmae_cv(y_cv, y_pred_cv_lr)) # Get CV error.
#         error_train_lr.append(wmae_train(y_train, y_pred_train_lr)) # Get Train error.
#         lr_hyperparams.append({'Fit Intercept':i, 'Normalize':j}) # Hyperparameters.

In [ ]:
# """Making dataframe containing values of hyper parameters with train and cv errors for the purpose of creating heatmaps."""

# lr_dataframe = pd.DataFrame(lr_hyperparams)
# lr_dataframe['Train Error'] = error_train_lr
# lr_dataframe['CV Error'] = error_cv_lr
# lr_dataframe.sort_values(by=['CV Error'], ascending=True)
# lr_dataframe

In [ ]:
# """Creating heatmaps for Train loss and CV loss."""

# sns.set(font_scale=1.3)
# train_lr = pd.pivot_table(lr_dataframe,'Train Error','Fit Intercept','Normalize') # Train Pivot table
# cv_lr = pd.pivot_table(lr_dataframe, 'CV Error','Fit Intercept','Normalize') # CV Pivot table.
# fig, ax = plt.subplots(1,2, figsize=(20,6))
# ax_train = sns.heatmap(train_lr, annot=True, fmt='4g', ax=ax[0]) # Train heatmap.
# ax_cv = sns.heatmap(cv_lr, annot=True, fmt='4g', ax=ax[1]) # CV heatmap.

# bottom_train, top_train = ax_train.get_ylim()
# ax_train.set_ylim(bottom_train + 0.5, top_train - 0.5)

# bottom_cv, top_cv = ax_cv.get_ylim()
# ax_cv.set_ylim(bottom_cv + 0.5, top_cv - 0.5)

# ax[0].set_title('Training set')
# ax[1].set_title('CV set')
# plt.show()

  - Observation: Taking fit_interceot=True and normalize=True as error values are same everywhere.
  

In [ ]:
# """Calculate Prediction and WMAE score."""

# model_linear_reg = LinearRegression(fit_intercept=True,normalize=True).fit(X_train,y_train) # Fit the model.
# y_pred = model_linear_reg.predict(X_test) # Predict test data.
# print('Weighted Mean Absolute Error (WMAE) for Linear Regression:', wmae_test(y_test, y_pred)) # Calculate WMAE score.

## 2.KNN Regression

In [ ]:
# # Define empty train and CV error lists and list of hyper parameter values. 
# error_cv = []
# error_train = []

# for i in range(3,22,2): # Loop over number of nearest neighbors (K) values [3,5,7,9,11,13,15,17,19,21].
#     knn = KNeighborsRegressor(n_neighbors=i) # Apply KNN Regressor.
#     knn.fit(X_train, y_train) # Fit the model.
#     y_pred_cv= knn.predict(X_cv) # Predict CV data.
#     y_pred_train= knn.predict(X_train) # Predict Train data.
#     error_cv.append(wmae_cv(y_cv, y_pred_cv)) # Get CV error.
#     error_train.append(wmae_train(y_train, y_pred_train)) # Get Train error.

In [ ]:
# """Plot Train error and CV error against number of nearest neighbors (K) values to select best hyper parameter."""

# plt.figure(figsize=(15, 6))
# plt.plot(range(3,22,2), error_cv, color='red', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10, label='CV Error')
# plt.plot(range(3,22,2), error_train, color='green', linestyle='dashed', marker='o',markerfacecolor='green', markersize=10, label='Train Error')
# plt.legend(loc='best')
# plt.title('Error Rate Vs K Value', fontsize=14)
# plt.xlabel('K Value',fontsize=12)
# plt.ylabel('Weighted Mean Absolute Error (WMAE)', fontsize=12)
# plt.show()

 - Observation: Best value of hyperparameter K is 11 as both training and CV error scores are low at K=11.

In [ ]:
# """Calculate Prediction and WMAE score."""

# model_knn = KNeighborsRegressor(n_neighbors=11).fit(X_train, y_train) # Fit the model with best K value.
# y_pred_test = model_knn.predict(X_test) # Predict test data.
# print('Weighted Mean Absolute Error (WMAE) for KNN Regression:', wmae_test(y_test, y_pred_test)) # Calculate WMAE score.

## 3.Decision Tree Regression

In [ ]:
# # Define the list of errors and list of hyper parameters.
# error_cv_dt = []
# error_train_dt = []
# max_depth = [1,5,10,15,20,25,30,35]
# min_samples_leaf = [1,2,3,4,5,6,7,8]
# dt_hyperparams = []

# """Calculating train and CV errors for maximum depth and minimum samples leaf parameters."""

# for i in max_depth: # Loop over max_depth.
#     for j in min_samples_leaf: # Loop over min_samples_leaf.
#         dt = DecisionTreeRegressor(max_depth=i, min_samples_leaf=j) # Apply Decision Tree Regressor.
#         dt.fit(X_train, y_train) # Fit the model.
#         y_pred_cv_dt = dt.predict(X_cv) # Predict CV data.
#         y_pred_train_dt = dt.predict(X_train) # Predict Train data.
#         error_cv_dt.append(wmae_cv(y_cv, y_pred_cv_dt)) # Calculate CV error.
#         error_train_dt.append(wmae_train(y_train, y_pred_train_dt)) # Calculate Train error.
#         dt_hyperparams.append({'depth':i, 'leaf':j}) # Get the list of hyper parameters.

In [ ]:
# """Making dataframe containing values of hyper parameters with train and cv errors for the purpose of creating heatmaps."""

# dt_dataframe = pd.DataFrame(dt_hyperparams)
# dt_dataframe['train error'] = error_train_dt
# dt_dataframe['cv error'] = error_cv_dt
# dt_dataframe.sort_values(by=['cv error'], ascending=True)
# dt_dataframe.head()

In [ ]:
# """Creating heatmaps for Train loss and CV loss."""

# sns.set(font_scale=1.1,)
# train_dt = pd.pivot_table(dt_dataframe ,'train error','depth','leaf') # Pivot table for train.
# cv_dt = pd.pivot_table(dt_dataframe , 'cv error','depth','leaf') # Pivot table for CV.
# fig, ax = plt.subplots(1,2, figsize=(20,6))
# sns.heatmap(train_dt, annot=True, fmt='2g', ax=ax[0]) # Train heatmap.
# sns.heatmap(cv_dt, annot=True, fmt='4g', ax=ax[1]) # CV heatmap.
# ax[0].set_title('Training set')
# ax[1].set_title('CV set')
# plt.show()

  - Observation: The Training loss and CV loss are least for maximum depth of 25 and minimum samples leaf of 5.

In [ ]:
# """Calculate Prediction and WMAE score."""

# model_dt = DecisionTreeRegressor(max_depth=25, min_samples_leaf=5).fit(X_train, y_train) # Fit the model with best hyper parameter values.
# y_pred = model_dt.predict(X_test) # Predict the test data.
# print('Weighted Mean Absolute Error (WMAE) for Decision Tree Regression:', wmae_test(y_test, y_pred)) # Get WMAE score.

### Random Forest Regression

In [ ]:
# # Define the list of errors and list of hyper parameters.
# error_cv_rf = []
# error_train_rf = []
# max_depth = [1,5,10,15,20,25,30,35]
# n_estimators = [10,20,30,40,50,60,70,80]
# rf_hyperparams = []

# """Calculating train and CV errors for maximum depth and number of estimators parameters."""

# for i in max_depth: # Loop over max_depth.
#     for j in n_estimators: # Loop over n_estimators. 
#         rf = RandomForestRegressor(max_depth=i, n_estimators=j) # Apply Random Forest Regressor.
#         rf.fit(X_train, y_train) # Fit the model.
#         y_pred_cv_rf = rf.predict(X_cv) # Predict CV data.
#         y_pred_train_rf = rf.predict(X_train) # Predict Train data.
#         error_cv_rf.append(wmae_cv(y_cv, y_pred_cv_rf)) # Get CV error.
#         error_train_rf.append(wmae_train(y_train, y_pred_train_rf)) # Get Train error.
#         rf_hyperparams.append({'Maximum Depth':i, 'No. of Estimators':j}) # Get list of hyper parameter values.

In [ ]:
# """Making dataframe containing values of hyper parameters with train and cv errors for the purpose of creating heatmaps."""

# rf_dataframe = pd.DataFrame(rf_hyperparams)
# rf_dataframe['Train Error']=error_train_rf
# rf_dataframe['CV Error']=error_cv_rf
# rf_dataframe.sort_values(by=['CV Error'], ascending=True)
# rf_dataframe.head()

In [ ]:
# """Creating heatmaps for Train loss and CV loss."""

# sns.set(font_scale=1.0)
# train_rf = pd.pivot_table(rf_dataframe,'Train Error','Maximum Depth','No. of Estimators') # Pivot table of Train data.
# cv_rf = pd.pivot_table(rf_dataframe, 'CV Error','Maximum Depth','No. of Estimators') # Pivot table of CV data.
# fig, ax = plt.subplots(1,2, figsize=(20,6))
# ax_train = sns.heatmap(train_rf, annot=True, fmt='2g', ax=ax[0], linewidths=0.01)
# ax_cv = sns.heatmap(cv_rf, annot=True, fmt='4g', ax=ax[1], linewidths=0.01)

# bottom_train, top_train = ax_train.get_ylim()
# ax_train.set_ylim(bottom_train + 0.5, top_train - 0.5)

# bottom_cv, top_cv = ax_cv.get_ylim()
# ax_cv.set_ylim(bottom_cv + 0.5, top_cv - 0.5)

# ax[0].set_title('Training set')
# ax[1].set_title('CV set')
# plt.show()

  - Observation: The training loss and CV loss are least for Maximum Depth of 35 and Number of Estimators of 80 .

In [ ]:
# """Calculate Prediction and  WMAE score."""

# model_rf = RandomForestRegressor(max_depth= 35, n_estimators=80).fit(X_train, y_train) # Fit the model with best hyper parameter values.
# y_pred = model_rf.predict(X_test) # Predict the test data.
# print('Weighted Mean Absolute Error (WMAE) for Random Forest Regression:', wmae_test(y_test, y_pred)) # Get WMAE score.

# Result

In [ ]:
models = pd.DataFrame({
    'Model Name': ['Linear Regression','KNN Regression','Decision Tree Regression'],
    
    'WMAE Score': ['14904.66', '11887.99', '2134.17']
    })

Index = pd.Series([1, 2, 3])
models.set_index(Index, inplace=True)
models

Test data Preprocess 

In [ ]:
#Reading kaggle provided test file for which the prediction is needed.
test_kaggle = pd.read_csv(data_dir + 'test.csv')
test_kaggle.head()

In [ ]:
test_kaggle = test_kaggle.merge(features, on=['Store', 'Date'], how='inner').merge(stores, on=['Store'], how='inner') # Joining test data with store and features.
print(test_kaggle.shape)

# Making IsHoliday as one column.
test_kaggle = test_kaggle.drop(['IsHoliday_y'], axis=1)
test_kaggle = test_kaggle.rename(columns={'IsHoliday_x':'IsHoliday'})
print('Test_Kaggle columns:\n', test_kaggle.columns)

# Converting Date to datetime
test_kaggle['Date'] = pd.to_datetime(test_kaggle['Date'])

# Extract date features
test_kaggle['Date_dayofweek'] = test_kaggle['Date'].dt.dayofweek
test_kaggle['Date_month'] = test_kaggle['Date'].dt.month 
test_kaggle['Date_year'] = test_kaggle['Date'].dt.year
test_kaggle['Date_day'] = test_kaggle['Date'].dt.day 


In [ ]:
test_kaggle_data = [test_kaggle]

# Converting Categorical Variable 'Type' into Numerical Variables.
type_mapping = {"A": 1, "B": 2, "C": 3}
for dataset in test_kaggle_data:
    dataset['Type'] = dataset['Type'].map(type_mapping)

# Converting Categorical Variable 'IsHoliday' into Numerical Variables.
type_mapping = {False: 0, True: 1}
for dataset in test_kaggle_data:
    dataset['IsHoliday'] = dataset['IsHoliday'].map(type_mapping)
#test = test.drop(['Date'], axis=1)


In [ ]:
# Special holidays.
test_kaggle['Super_Bowl'] = np.where((test_kaggle['Date'] == datetime(2010,2,10)) | (test_kaggle['Date'] == datetime(2011,2,11)) | 
                               (test_kaggle['Date'] == datetime(2012,2,10)) | (test_kaggle['Date'] == datetime(2013,2,8)), 1, 0)
test_kaggle['Labor_day'] = np.where((test_kaggle['Date'] == datetime(2010,9,10)) | (test_kaggle['Date'] == datetime(2011,9,9)) | 
                              (test_kaggle['Date'] == datetime(2012,9,7)) | (test_kaggle['Date'] == datetime(2013,9,6)), 1, 0)
test_kaggle['Thanksgiving'] = np.where((test_kaggle['Date']==datetime(2010, 11, 26)) | (test_kaggle['Date']==datetime(2011, 11, 25)) | 
                                 (test_kaggle['Date']==datetime(2012, 11, 23)) | (test_kaggle['Date']==datetime(2013, 11, 29)),1,0)
test_kaggle['Christmas'] = np.where((test_kaggle['Date']==datetime(2010, 12, 31)) | (test_kaggle['Date']==datetime(2011, 12, 30)) | 
                              (test_kaggle['Date']==datetime(2012, 12, 28)) | (test_kaggle['Date']==datetime(2013, 12, 27)),1,0)

In [ ]:
# Since we have Imputed IsHoliday according to Extra holidays..These extra holiday variable has redundant.
# Droping the Extra holiday variables because its redundant.
dp = ['Super_Bowl','Labor_day','Thanksgiving','Christmas']
test_kaggle.drop(dp, axis=1, inplace=True)

In [ ]:
test_kaggle = test_kaggle.fillna(0) # Filling null values with 0.

In [ ]:
test_kaggle.info()

In [ ]:
# Removing unimportant features.
# features_drop = ['Unemployment','CPI','MarkDown5']
features_drop = ['Date_year','CPI','MarkDown1']
test_kaggle = test_kaggle.drop(features_drop, axis=1)

In [ ]:
#Excluding Date as it throws error while making prediction.
test_kaggle = test_kaggle.loc[:, test_kaggle.columns != 'Date']

In [ ]:
print('Final test_kaggle shape:', test_kaggle.shape)
test_kaggle.head(2)

In [ ]:
# # Applying Random Forest to kaggle provided test file with the best hyper parameter values we got during training phase in Random Forest.
# y_ = train['Weekly_Sales']
# X_ = train.drop(['Weekly_Sales', 'Date'], axis=1) 
# X_ = X_.drop(features_drop, axis=1)

# model_rf = RandomForestRegressor(max_depth= 35, n_estimators=80).fit(X_, y_) # Fit the model with original train data.
# y_pred = model_rf.predict(test_kaggle) # Predict the final test data that Kaggle has provided.

In [ ]:
y_ = train['Weekly_Sales']
X_ = train.drop(['Weekly_Sales', 'Date'], axis=1)
X_ = X_.drop(features_drop, axis=1)

model_dt = DecisionTreeRegressor(max_depth=25, min_samples_leaf=5).fit(X_, y_) 
y_pred = model_dt.predict(test_kaggle)  

In [ ]:
test_kaggle

In [ ]:
y_pred

**Submission File**

In [ ]:
test_kaggle_ = pd.read_csv(data_dir+'test.csv')
print(test_kaggle_.shape)
test_kaggle_.head()

In [ ]:
submission = pd.DataFrame({
        "Id": test_kaggle_.Store.astype(str)+'_'+test_kaggle_.Dept.astype(str)+'_'+test_kaggle_.Date.astype(str),
        "Weekly_Sales": y_pred 
    })

submission.to_csv('WeeklySalesPrediction_DT.csv', index=False) 

In [ ]:
submission

In [ ]:
test_kaggle[115061:]

In [ ]:
submission[115061:]